In [ ]:
# ===== Rough Flask App #2: Tiny Web UI + CSV Log =====
# Self-contained HTML using render_template_string (no external templates needed).
# Logs each request to data/requests_log.csv (time, dish, age, disease, trust_score).

!pip install -q flask flask-ngrok

import os
from datetime import datetime
from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok
from mistral_llm_cpu import generate_recipe_output_with_trust, all_diseases

app_ui = Flask(__name__)
run_with_ngrok(app_ui)

TEMPLATE_080 = """
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>NutriFusion Mini UI</title>
  <style>
    body { font-family: system-ui, Arial; margin: 2rem; }
    .wrap { max-width: 720px; margin: 0 auto; }
    form { display: grid; gap: 10px; padding: 16px; border: 1px solid #ddd; border-radius: 12px; }
    label { font-weight: 600; }
    input, select, textarea { padding: 8px; width: 100%; }
    button { padding: 10px 14px; font-weight: 700; border-radius: 8px; }
    .row { display: grid; grid-template-columns: 1fr 1fr; gap: 10px; }
    .card { border:1px solid #eee; border-radius:12px; padding:16px; margin-top:16px; }
    small { opacity: .7; }
    pre { white-space: pre-wrap; }
  </style>
</head>
<body>
  <div class="wrap">
    <h1>NutriFusion — Quick Tester <small>(rough)</small></h1>
    <form method="post" action="/">
      <div class="row">
        <div>
          <label>Dish</label>
          <input name="dish" value="{{ dish or 'Grilled Chicken Salad' }}" required>
        </div>
        <div>
          <label>Age</label>
          <input name="age" type="number" value="{{ age or 35 }}" required>
        </div>
      </div>
      <div>
        <label>Disease</label>
        <select name="disease">
          {% for d in diseases %}
            <option value="{{ d }}" {% if disease==d %}selected{% endif %}>{{ d }}</option>
          {% endfor %}
        </select>
      </div>
      <button type="submit">Generate</button>
    </form>

    {% if output is defined %}
      <div class="card">
        <h3>Output</h3>
        <pre>{{ output }}</pre>
        <p><strong>Trust score:</strong> {{ trust_score }}</p>
        <details>
          <summary>Explanation</summary>
          <pre>{{ explanation }}</pre>
        </details>
      </div>
    {% endif %}

    <div class="card">
      <h3>About</h3>
      <p>This is a minimal UI that calls <code>generate_recipe_output_with_trust()</code> and logs
         each request to <code>data/requests_log.csv</code> for debug purposes.</p>
    </div>
  </div>
</body>
</html>
"""

LOG_PATH_080 = "data/requests_log.csv"
os.makedirs("data", exist_ok=True)
if not os.path.exists(LOG_PATH_080):
    with open(LOG_PATH_080, "w", encoding="utf-8") as f:
        f.write("timestamp,dish,age,disease,trust_score\n")

@app_ui.route("/", methods=["GET", "POST"])
def index_080():
    ctx = {"diseases": all_diseases}
    if request.method == "POST":
        dish = request.form.get("dish", "").strip()
        age = int(request.form.get("age", 0))
        disease = request.form.get("disease", "").strip()

        output, _, trust_score, explanation = generate_recipe_output_with_trust(dish, age, disease)
        ctx.update({
            "dish": dish, "age": age, "disease": disease,
            "output": output, "trust_score": trust_score, "explanation": explanation
        })

        # append a simple CSV log row
        with open(LOG_PATH_080, "a", encoding="utf-8") as f:
            f.write(f"{datetime.now().isoformat()},{dish},{age},{disease},{trust_score}\n")

    return render_template_string(TEMPLATE_080, **ctx)

app_ui.run()
